<a href="https://colab.research.google.com/github/aquibjaved/Bits_and_Pieces_DL/blob/main/Basics_of_lora3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/cccntu/minLoRA.git
!cd minLoRA
!pip install -e /content/minLoRA/

Cloning into 'minLoRA'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 42 (delta 21), reused 36 (delta 16), pack-reused 0
Receiving objects: 100% (42/42), 15.04 KiB | 2.15 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Obtaining file:///content/minLoRA
  Preparing metadata (setup.py) ... done
  Running setup.py develop for minLoRA


In [1]:
import torch
from torch import nn
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.nn.utils.parametrize as P
from sklearn.datasets import load_iris
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
import math

from minlora import add_lora, apply_to_lora, disable_lora, enable_lora, get_lora_params, merge_lora, name_is_lora, remove_lora, load_multiple_lora, select_lora, get_lora_state_dict

In [ ]:
# model = FashionNet(input_dims=784, num_class=10, input_hidden_size=1000, hidden_layer_size=2000)
# Step 1: Add LoRA to the model
# add_lora(model)

# Step 2: Collect the parameters, pass them to the optimizer

# parameters = [
#     {"params": list(get_lora_params(model))},
# ]
# optimizer = torch.optim.AdamW(parameters, lr=1e-3)

# Step 3: Train the model
# ...

# Step 4: export the LoRA parameters
# lora_state_dict = get_lora_state_dict(model)

In [2]:
# Define a transform to normalize the data and flatten each image into a 1D array
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(lambda x: x.view(-1))
])

def collate_fn(batch):
    data, target = zip(*batch)
    data = torch.stack(data)
    target = torch.tensor(target)
    return data.cuda(), target.cuda()

train_dataset = datasets.FashionMNIST('~/.data', download=True, train=True, transform=transform)
test_dataset = datasets.FashionMNIST('~/.data', download=True, train=False, transform=transform)
def split_dataset(dataset, labels):
    indices = [i for i, (data, label) in enumerate(dataset) if label in labels]
    return torch.utils.data.Subset(dataset, indices)

labels_1 = [0, 1, 2, 3, 4]
labels_2 = [5, 6, 7, 8, 9]

train_dataset_04 = split_dataset(train_dataset, labels_1)
train_dataset_59 = split_dataset(train_dataset, labels_2)

batch_size = 64

train_loader_1 = torch.utils.data.DataLoader(train_dataset_04, batch_size=batch_size, shuffle=True)
train_loader_2 = torch.utils.data.DataLoader(train_dataset_59, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26421880/26421880 [00:00<00:00, 116904893.84it/s]


Extracting /root/.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 11135637.54it/s]

Extracting /root/.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 63605356.93it/s]


Extracting /root/.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 25492652.88it/s]


Extracting /root/.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.data/FashionMNIST/raw



In [3]:
import torch

def train_model(epochs:int, train_loader, model, optimizer, criterion):
  model.train()
  train_loss_per_epoch = []

  for epoch in range(epochs):

    running_loss_train = 0.0

    for data_batch, target_batch in train_loader:

        data_batch = data_batch.to('cuda')
        target_batch = target_batch.to('cuda')

        optimizer.zero_grad()
        output = model(data_batch)
        loss_train = criterion(output, target_batch)
        loss_train.backward()
        optimizer.step()
        running_loss_train += loss_train.item()

    epcoh_loss_train = running_loss_train / len(train_loader)

    train_loss_per_epoch.append(epcoh_loss_train)


    if epoch %2 == 0:
      print(f'Epoch {epoch} Train loss {epcoh_loss_train}')
  return train_loss_per_epoch

device='cuda'

import torch
from sklearn.metrics import confusion_matrix
import numpy as np

def test(test_loader, net):
    correct = 0
    total = 0

    wrong_counts = [0 for i in range(10)]

    with torch.no_grad():
        for data in test_loader:
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = net(x)
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct +=1
                else:
                    wrong_counts[y[idx]] +=1
                total +=1
    print(f'Accuracy: {round(correct/total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the Label {i}: {wrong_counts[i]}')



In [4]:
class FashionNet(nn.Module):
  def __init__(self, input_dims, num_class, input_hidden_size: 1000, hidden_layer_size:2000):

    super().__init__()
    self.input_dims = input_dims
    self.num_class = num_class

    self.input_layer = nn.Linear(in_features=self.input_dims, out_features=1000)
    self.hidden_layer = nn.Linear(in_features=1000, out_features=2000)
    self.output_layer = nn.Linear(in_features=2000, out_features=self.num_class)

  def forward(self, x):
    out = F.relu(self.input_layer(x))
    out = F.relu(self.hidden_layer(out))
    out = self.output_layer(out)
    return out

In [5]:
cuda_available = torch.cuda.is_available()

# Define the class names for Fashion MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

BigNet = FashionNet(input_dims=784, num_class=len(class_names), input_hidden_size=1000, hidden_layer_size=2000)
optimizer = optim.SGD(BigNet.parameters(), lr=0.0001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Assume `model` is your model
if cuda_available:
    BigNet = BigNet.to('cuda')
    optimizer = optim.SGD(BigNet.parameters(), lr=0.011, momentum=0.9)
    criterion = nn.CrossEntropyLoss().to('cuda')

is_on_gpu = next(BigNet.parameters()).is_cuda
print(f'Model is on GPU: {is_on_gpu}')

trainable_params = sum(p.numel() for p in BigNet.parameters() if p.requires_grad)
print(f'Number of trainable (tunable) parameters: {trainable_params}')

Model is on GPU: True
Number of trainable (tunable) parameters: 2807010


In [6]:
trainLoss = train_model(epochs=10,
                        train_loader=train_loader_1,
                        optimizer = optimizer,
                        criterion=criterion,
                        model=BigNet,
                        )

Epoch 0 Train loss 0.4198003794783468
Epoch 2 Train loss 0.264796864948293
Epoch 4 Train loss 0.2255258931756528
Epoch 6 Train loss 0.1997014654280026
Epoch 8 Train loss 0.17623552429015193


In [7]:
test(
      test_loader=test_loader,
      net=BigNet,
)

Accuracy: 0.454
wrong counts for the Label 0: 71
wrong counts for the Label 1: 37
wrong counts for the Label 2: 117
wrong counts for the Label 3: 64
wrong counts for the Label 4: 172
wrong counts for the Label 5: 1000
wrong counts for the Label 6: 1000
wrong counts for the Label 7: 1000
wrong counts for the Label 8: 1000
wrong counts for the Label 9: 1000


In [8]:
lora_model = BigNet.to('cpu')
# Step 1: Add LoRA to the model
add_lora(lora_model)

# Step 2: Collect the parameters, pass them to the optimizer

parameters = [
    {"params": list(get_lora_params(lora_model))},
]
optimizer = optim.SGD(BigNet.parameters(), lr=0.0001, momentum=0.9)

# Step 3: Train the model
# ...

# Step 4: export the LoRA parameters
# lora_state_dict = get_lora_state_dict(model)


In [9]:
lora_model.to('cuda')

FashionNet(
  (input_layer): ParametrizedLinear(
    in_features=784, out_features=1000, bias=True
    (parametrizations): ModuleDict(
      (weight): ParametrizationList(
        (0): LoRAParametrization()
      )
    )
  )
  (hidden_layer): ParametrizedLinear(
    in_features=1000, out_features=2000, bias=True
    (parametrizations): ModuleDict(
      (weight): ParametrizationList(
        (0): LoRAParametrization()
      )
    )
  )
  (output_layer): ParametrizedLinear(
    in_features=2000, out_features=10, bias=True
    (parametrizations): ModuleDict(
      (weight): ParametrizationList(
        (0): LoRAParametrization()
      )
    )
  )
)

In [10]:
trainLoss = train_model(epochs=10,
                        train_loader=train_loader_2,
                        optimizer = optimizer,
                        criterion=criterion,
                        model=lora_model,
                        )

Epoch 0 Train loss 3.0382540015015267
Epoch 2 Train loss 0.8434577175040743
Epoch 4 Train loss 0.46272345780055407
Epoch 6 Train loss 0.37157765259620734
Epoch 8 Train loss 0.32912180024677756


In [ ]:
# Call the function
test(
      test_loader=test_loader,
      net=lora_model,
)

Accuracy: 0.517
wrong counts for the Label 0: 1000
wrong counts for the Label 1: 279
wrong counts for the Label 2: 998
wrong counts for the Label 3: 1000
wrong counts for the Label 4: 1000
wrong counts for the Label 5: 181
wrong counts for the Label 6: 51
wrong counts for the Label 7: 133
wrong counts for the Label 8: 88
wrong counts for the Label 9: 96


In [11]:
lora_state_dict = get_lora_state_dict(lora_model)

original_model = BigNet.to('cpu')
add_lora(lora_model)
_ = original_model.load_state_dict(lora_state_dict, strict=False) # original model with lora trained weights


In [12]:
merge_lora(original_model)


In [13]:
original_model

FashionNet(
  (input_layer): Linear(in_features=784, out_features=1000, bias=True)
  (hidden_layer): Linear(in_features=1000, out_features=2000, bias=True)
  (output_layer): Linear(in_features=2000, out_features=10, bias=True)
)

In [15]:
# Call the function
test(
      test_loader=test_loader,
      net=original_model.to('cuda'),
)

Accuracy: 0.518
wrong counts for the Label 0: 1000
wrong counts for the Label 1: 280
wrong counts for the Label 2: 998
wrong counts for the Label 3: 1000
wrong counts for the Label 4: 1000
wrong counts for the Label 5: 155
wrong counts for the Label 6: 47
wrong counts for the Label 7: 160
wrong counts for the Label 8: 89
wrong counts for the Label 9: 92
